In [ ]:
import chromedriver_autoinstaller
from selenium import webdriver
import pandas as pd
import re
import math
import time
import os, sys
import requests
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.action_chains import ActionChains


In [ ]:
def open_chromedriver():
    chrome = chromedriver_autoinstaller.install(os.getcwd())

    options = webdriver.ChromeOptions()
    options.headless = True

    driver = webdriver.Chrome(options=options)
    return(driver)


In [ ]:
def get_tch_home():
    link=[]
    for i in driver.find_elements_by_class_name('tlistarea'):
        for j in i.find_elements_by_tag_name('li'):
            try:
                link.append(j.find_element_by_tag_name('a').get_attribute('href'))
            except:
                break
    link=list(set(link))
    print(len(link))
    return(link)

In [ ]:
def get_lec_link(tch_home):
    lec_link=[]
    num=0
    for j in tch_home:
        num=num+1
        
        print(num,'/',len(tch_home))
        
        driver.get(j)
        
        time.sleep(1)
        
        for i in driver.find_elements_by_xpath('//*[@id="tcherLctrList"]/div/div[4]/div[*]/table/tbody/tr/td[3]/a'):
            lec_link.append(i.get_attribute('href'))
    return(lec_link)

In [ ]:
#과목
def get_sub():
    sub=re.split('\n',soup.find('div',{'id':'tcdHome'}).find('h2').get_text())[0]
    return(sub)

In [ ]:
#선생님 이름
def get_tch_name():
    tch_name=soup.find('strong',{'class':'teacher_home'}).find('span').text
    return(tch_name)

In [ ]:
#선생님 id
def get_tch_id():
    tch_id=re.split('tcd=',soup.find('strong',{'class':'teacher_home'}).find('a')['href'])[1]
    return(tch_id)

In [ ]:
#강의 id
def get_lec_id(i):
    lec_id=re.split('=',i)[1]
    return(lec_id)

In [ ]:
#강의 제목
def get_lec_title():
    lec_title = soup.find('div',{'class':'titarea'}).find('h3').text
    return(lec_title)

In [ ]:
#강의 내용 가져오기

def get_data_2():
    data_2=[]
    list3 =[]
    temp=[]
    
    list2=[]
    for t in soup.select('#crclmBody > tr> td.tl'):
        list2.append(t.text)
        
    for t in soup.select('#crclmBody > tr > td'):
        temp.append(t.text)

    for t in temp:
        if len(re.findall('[0-9]분',t))!=0:
            list3.append(t)

    list1 = list(range(1,len(list2)+1))

    data_2.append([','.join(['(' + str(a) + ', ' + b+', '+c+')' for a,b,c in zip(list1,list2,list3)])])
    return(data_2)

In [ ]:
# 완강 여부
def get_complete():
    try:
        if re.split('/ ',driver.find_element_by_xpath('//*[@id="lecturedetailDiv"]/div[3]/div[1]/div[2]/div[1]/ul/li[3]').text)[1] ==['제작중']:
            complete='X'
        else :
            complete='O'
    except:
        complete='표기없음'
    return(complete)

In [ ]:
driver= open_chromedriver()

In [ ]:
driver.get('http://www.mimacstudy.com/tcher/home/tcherHomeMain.ds?requestMenuId=MNMN_M004')

In [ ]:
time.sleep(3)

In [ ]:
tch_home = get_tch_home()

In [ ]:
lec_link = get_lec_link(tch_home)

In [ ]:
sub=[]
tch_name=[]
tch_id=[]
lec_id=[]
lec_title=[]
data_2=[]
complete=[]
num=0
for i in lec_link:
    driver.get(i)
    
    num=num+1
    print(num,'/',len(lec_link))
    
    time.sleep(2)
    
    data = driver.page_source
    
    soup = bs(data, 'html.parser')
    
    
    
    sub.append(get_sub())
    tch_name.append(get_tch_name())
    tch_id.append(get_tch_id())
    lec_id.append(get_lec_id(i))
    lec_title.append(get_lec_title())
    complete.append(get_complete())
    data_2.extend(get_data_2())
    

In [ ]:
pd.DataFrame({'과목':sub,'선생님':tch_name,'선생님ID':tch_id,'강좌명':lec_title,'강좌ID':lec_id,'완강여부':complete,'강의번호&챕터명&시간':data_2})